In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install tensorflow
!pip install keras


In [ ]:
!pip install scipy
!pip install glob2

In [ ]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob

In [ ]:
IMAGESHAPE = [224, 224, 3]
training_data = '/content/drive/MyDrive/Image-dataset-pneumonia/train'
testing_data = '/content/drive/MyDrive/Image-dataset-pneumonia/test'

In [ ]:
import os

# Path to the directory in Google Drive
directory_path_normal = '/content/drive/MyDrive/Image-dataset-pneumonia/train/NORMAL'
directory_path_pneumonia = '/content/drive/MyDrive/Image-dataset-pneumonia/train/PNEUMONIA'

# Count the number of files in the directory
num_files = sum([len(files) for r, d, files in os.walk(directory_path_normal)])
print(f'Normal count "{directory_path_normal}" is: {num_files}')

num_files = sum([len(files) for r, d, files in os.walk(directory_path_pneumonia)])
print(f'Pneumonia count "{directory_path_pneumonia}" is: {num_files}')


Normal count "/content/drive/MyDrive/Image-dataset-pneumonia/train/NORMAL" is: 1341
Pneumonia count "/content/drive/MyDrive/Image-dataset-pneumonia/train/PNEUMONIA" is: 3875


In [ ]:
# from PIL import Image

# def verify_images(directory):
#     for subdir, dirs, files in os.walk(directory):
#         for file in files:
#             file_path = os.path.join(subdir, file)
#             try:
#                 img = Image.open(file_path)
#                 img.verify()  # Verify that it is, in fact, an image
#             except (IOError, SyntaxError) as e:
#                 print(f"Removing corrupted image: {file_path}")
#                 os.remove(file_path)

# # Verify and clean the training and testing directories
# verify_images('/content/drive/MyDrive/Dataset/train')
# verify_images('/content/drive/MyDrive/Dataset/test')

In [ ]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
for each_layer in vgg_model.layers:
    each_layer.trainable = False

In [ ]:
# classes = glob('/content/drive/MyDrive/Dataset/train*')

In [ ]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(1, activation='sigmoid')(flatten_layer)

In [ ]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
import tensorflow as tf
final_model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=[tf.keras.metrics.Recall()]
)

In [ ]:

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)

In [ ]:

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Image-dataset-pneumonia/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 5216 images belonging to 2 classes.


In [ ]:
test_set = testing_datagen.flow_from_directory('/content/drive/MyDrive/Image-dataset-pneumonia/test',
                                               target_size = (224, 224),
                                               batch_size = 32,
                                               class_mode = 'binary')

Found 624 images belonging to 2 classes.


In [ ]:

fitted_model = final_model.fit(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=163,
  validation_steps=20
)

Epoch 1/10
163/163 [==============================] - 539s 3s/step - loss: 0.1202 - recall: 0.9698 - val_loss: 0.3984 - val_recall: 0.9923
Epoch 2/10
163/163 [==============================] - 135s 826ms/step - loss: 0.1092 - recall: 0.9734 - val_loss: 0.3939 - val_recall: 0.9949
Epoch 3/10
163/163 [==============================] - 136s 837ms/step - loss: 0.0933 - recall: 0.9757 - val_loss: 0.4189 - val_recall: 0.9949
Epoch 4/10
163/163 [==============================] - 143s 878ms/step - loss: 0.0787 - recall: 0.9794 - val_loss: 0.2611 - val_recall: 0.9641
Epoch 5/10
163/163 [==============================] - 139s 853ms/step - loss: 0.0837 - recall: 0.9794 - val_loss: 0.2284 - val_recall: 0.9410
Epoch 6/10
163/163 [==============================] - 141s 864ms/step - loss: 0.0752 - recall: 0.9819 - val_loss: 0.2386 - val_recall: 0.9487
Epoch 7/10
163/163 [==============================] - 139s 848ms/step - loss: 0.0657 - recall: 0.9837 - val_loss: 0.2800 - val_recall: 0.9821
Epoch 8/1

In [ ]:
final_model.save('/content/drive/MyDrive/saved_models/our_model.h5') #Saving the model file.


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
from tensorflow.keras.models import load_model

In [3]:
model = load_model('/content/drive/MyDrive/saved_models/pneumonia-model.h5')

In [16]:
testing_image = '/content/drive/MyDrive/Image-dataset-pneumonia/val/PNEUMONIA/person1951_bacteria_4882.jpeg'

In [17]:
from tensorflow.keras.preprocessing import image
import numpy as np
# Load the image with target size matching the input size of the model
img = image.load_img(testing_image, target_size=(224, 224))  # Adjust target_size as per your model input size

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Expand the dimensions to match the model's input shape
img_array = np.expand_dims(img_array, axis=0)

# Normalize the image data (assuming the model was trained on normalized data)
img_array = img_array / 255.0

In [18]:
if model.predict(img_array)[0][0] > 0.5:
  print('The person is affected by pneumonia')
else:
  print('The person is safe')

1/1 [==============================] - 1s 514ms/step
The person is affected by pneumonia
